In [1]:
import wrds
conn = wrds.Connection(wrds_username='fanjia')

Loading library list...
Done


In [90]:
import pandas as pd

In [11]:
######################################## Testing IBES
x = conn.raw_sql(
    """ SELECT surprise.ticker
        FROM ibes.surpsum AS surprise
        WHERE surprise.anndats >= '2001-01-01'::date
        ORDER BY surprise.anndats;
    """)
x

,ticker
0,@G2R
1,@G2R
2,@G2R
3,@G2R
4,@G2R
...,...
9173598,ZSAN
9173599,ZSAN
9173600,ZSAN
9173601,ZSAN


In [2]:
df = conn.raw_sql(
    """ SELECT
        datadate AS date,
        gvkey,
        loc AS country,
        indfmt AS industry,
        sich AS classification,
        at,
        ppegt,
        invt,
        act,
        chee,
        ivao,
        ivst,
        intan,
        gdwl,
        ceq,
        lt,
        lct,
        dlc,
        capx,
        icapt,
        revt,
        xrd,
        ebit,
        ebitda,
        nicon,
        oancf,
        COALESCE(txp, txpfs) as txp, -- =tp
        sstk,
        prstkc,
        ivaeq,
        dvt,
        dv,
        dp,
        dltt,
        pstk,
        at - che AS oa,
        act - che AS coa,
        lct - dlc AS col,
        at - dlc - dltt - mib - pstk - ceq AS ol,
        act - che - lct + dlc AS wc,
        at - act - COALESCE(ivao, 0) AS ncoa,
        lt - lct - dltt AS ncol,
        at - act - COALESCE(ivao, 0) - (lt - lct - dltt) AS nncoa,
        ivst + ivao AS fa,
        dltt + dlc + pstk AS fl,
        ivst + ivao - (dltt + dlc + pstk) AS nfa,
        (nicon - oancf) / NULLIF(ABS(nicon), 0) AS poa,
        (nicon - (- sstk + prstkc + dv + oancf + ivncf + fincf)) / NULLIF(nicon, 0) AS pta,
        at - act - COALESCE(ivao, 0) + act - che - lct + dlc + ivst + ivao - (dltt + dlc + pstk) AS ta,
        act / NULLIF(lct, 0) AS curr_ratio,
        (act - invt) / NULLIF(lct, 0) AS quick_ratio,
        chee / NULLIF(lct, 0) AS cash_ratio,
        oancf / NULLIF(lct, 0) AS opr_cashflow_ratio,
        capx / NULLIF(oancf, 0) AS capx_cashflow_ratio,
        (dlc + dltt) / NULLIF(ceq, 0) AS debt_equity_ratio,
        (dltt + dlc) / NULLIF(at, 0) debt_asset_ratio,
        at / NULLIF(ceq, 0) AS fin_leverage,
        (dltt + dlc) / NULLIF(ebitda, 0) AS debt_ebitda_ratio,
        (dltt + dlc - chee) / NULLIF(ebitda, 0) AS net_debt_ebitda_ratio,
        ebit / NULLIF(xint, 0) AS ebit_int_coverage_ratio,
        ebitda / NULLIF(xint, 0) AS ebitda_int_coverage_ratio,
        ch / NULLIF(xint, 0) AS cash_coverage_ratio,
        (dltt + dlc) / NULLIF(dltt + dlc + teq, 0) AS debt_total_cap_ratio,
        oancf / NULLIF(dlc + dltt, 0) AS cashflow_debt_ratio,
        cogs / NULLIF(invt, 0) AS invt_turnover,
        cogs / NULLIF(ap, 0) AS accounts_payable_turnover,
        revt / NULLIF(COALESCE(rect, artfs), 0) AS rec_turnover,
        365 * invt / NULLIF(cogs, 0) AS days_sales_invt,
        365 * ap / NULLIF(cogs, 0) AS days_sales_payable,
        365 * NULLIF(COALESCE(rect, artfs), 0) / NULLIF(revt, 0) AS days_sales_receivable,
        revt / NULLIF(ppent, 0) AS fixed_asset_turnover,
        revt / NULLIF(at, 0) AS total_asset_turnover,
        (revt - cogs) / NULLIF(revt, 0) AS gross_profit_margin,
        ebitda / NULLIF(revt, 0) AS ebitda_margin,
        ebit / NULLIF(revt, 0) AS ebit_margin,
        pi / NULLIF(revt, 0) AS pre_tax_margin,
        nicon / NULLIF(revt, 0) AS net_profit_margin,
        nicon / NULLIF(at, 0) AS roa,
        nicon / NULLIF(ceq, 0) AS roe,
        COALESCE((ebit * (nicon / NULLIF(pi, 0))) / NULLIF(dlc + dltt + teq, 0), ebit * (1 - txt / NULLIF(pi, 0)) / NULLIF(icapt, 0)) AS roic,
        COALESCE(epsexcon, nicon / NULLIF(cshpria, 0)) AS eps,
        COALESCE(epsincon, (nicon + xido) / NULLIF(cshpria, 0)) AS eps_inextra,
        fincf + ivncf + oancf AS cashflow,
        xrd / NULLIF(revt, 0) AS rd_sales_ratio
FROM
    comp_global_daily.g_funda
WHERE 
    exchg = ANY (ARRAY [104, 107, 132, 151, 154, 171, 192, 194, 201, 209, 256, 257, 273, 276, 286])
    AND curcd = 'EUR';

        """)


In [78]:
##### Accruals (Acc)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','act','chee','lct','dlc','txp','dp','at']]

df2.loc[max(df2.index)+1, :] = None

df2['change_act'] = df2['act'] - df2['act'].shift(1)
df2['change_chee'] = df2['chee'] - df2['chee'].shift(1)
df2['change_lct'] = df2['lct'] - df2['lct'].shift(1)
df2['change_dlc'] = df2['dlc'].fillna(0) - df2['dlc'].shift(1).fillna(0)
df2['change_txp'] = df2['txp'] - df2['txp'].shift(1)
df2['change_chee'] = df2['chee'] - df2['chee'].shift(1)
df2['avg_at'] = (df2['at'] + df2['at'].shift(1))/2

acc = ((df2['change_act'] - df2['change_chee']) - (df2['change_lct'] - df2['change_dlc'] - df2['change_txp']) - df2['dp'])/df2['avg_at']
acc

0          NaN
1     0.030740
2    -0.040623
3    -0.055401
4    -0.123233
5    -0.004306
6     0.008315
7    -0.054317
8     0.013350
9    -0.056480
10   -0.150775
11    0.058354
12    0.091667
13   -0.006740
14   -0.279665
15   -0.026405
16   -0.027095
17   -0.008466
18   -0.015259
19   -0.014258
20   -0.054996
21    0.001935
22         NaN
dtype: float64

In [79]:
##### Change in Current Operating Assets (ChCOA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','coa','at']]

df2.loc[max(df2.index)+1, :] = None

chcoa = (df2['coa'] - df2['coa'].shift(1))/df2['at'].shift(1)
chcoa

0          NaN
1     0.419283
2    -0.106170
3    -0.035421
4    -0.043586
5     0.066503
6     0.083223
7    -0.003899
8     0.051644
9    -0.075220
10   -0.062457
11    0.281073
12    0.174344
13    0.007821
14   -0.408335
15    0.017160
16   -0.005014
17    0.023852
18    0.025017
19    0.031414
20    0.028522
21    0.031416
22         NaN
dtype: float64

In [85]:
##### Change in Current Operating Liabilities (ChCOL)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','col','at']]

df2.loc[max(df2.index)+1, :] = None

chcol = (df2['col'].fillna(0) - df2['col'].shift(1))/df2['at'].shift(1)
chcol

0          NaN
1     0.274328
2    -0.145178
3    -0.034984
4     0.026938
5     0.025654
6     0.022254
7     0.014742
8     0.001095
9    -0.053762
10    0.039647
11    0.203251
12    0.036174
13   -0.038813
14   -0.165758
15    0.021504
16   -0.010844
17    0.001337
18    0.015626
19    0.019663
20    0.064571
21   -0.144164
22         NaN
dtype: float64

In [86]:
##### Change in Net Non-Cash Working Capital (ChNNCWC)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','wc','at']]

df2.loc[max(df2.index)+1, :] = None

chnncwc = (df2['wc'].fillna(0) - df2['wc'].shift(1))/df2['at'].shift(1)
chnncwc

0          NaN
1     0.144955
2     0.039008
3    -0.000437
4    -0.070523
5     0.040849
6     0.060969
7    -0.018641
8     0.050549
9    -0.021458
10   -0.102104
11    0.077822
12    0.138170
13    0.046634
14   -0.242577
15   -0.004344
16    0.005830
17    0.022515
18    0.009391
19    0.011751
20   -0.036049
21   -0.061535
22         NaN
dtype: float64

In [98]:
##### Change in Net Non-Current Operating Assets (ChNNCOA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','ncoa','at']]

df2.loc[max(df2.index)+1, :] = None

chnncoa = (df2['ncoa'].fillna(0) - df2['ncoa'].shift(1))/df2['at'].shift(1)
chnncoa #drop the last row 22th

0          NaN
1     0.206447
2     0.078344
3    -0.079393
4    -0.061957
5     0.099586
6     0.004286
7    -0.043969
8    -0.010811
9     0.001572
10   -0.006211
11    0.089979
12    0.071224
13    0.002961
14    0.441533
15    0.113279
16    0.107868
17    0.044089
18   -0.224883
19    0.070241
20    0.033817
21    0.022578
22   -0.573537
dtype: float64

In [114]:
##### Change in Non-Current Operating Assets (ChNCOA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at','act','ivao']]

df2.loc[max(df2.index)+1, :] = None

chncoa = ((df2['at']-df2['act']-df2['ivao'].fillna(0)) - (df2['at']-df2['act']-df2['ivao'].fillna(0)).shift(1))/df2['at'].shift(1)
chncoa

0          NaN
1     0.206447
2     0.078344
3    -0.079393
4    -0.061957
5     0.099586
6     0.004286
7    -0.043969
8    -0.010811
9     0.001572
10   -0.006211
11    0.089979
12    0.071224
13    0.002961
14    0.441533
15    0.113279
16    0.107868
17    0.044089
18   -0.224883
19    0.070241
20    0.033817
21    0.022578
22         NaN
dtype: float64

In [117]:
##### Change in Non-Current Operating Liabilities (ChNCOL)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at','lt','lct','dltt']]

df2.loc[max(df2.index)+1, :] = None

chncol = ((df2['lt']-df2['lct']-df2['dltt']) - (df2['lt']-df2['lct']-df2['dltt']).shift(1))/df2['at'].shift(1)
chncol

0          NaN
1    -0.006239
2     0.001464
3    -0.001224
4     0.001055
5    -0.001229
6    -0.000732
7     0.004677
8     0.000674
9     0.002806
10    0.027380
11   -0.024143
12    0.008315
13    0.000763
14   -0.010897
15   -0.000403
16    0.005844
17    0.000980
18   -0.000133
19   -0.000955
20    0.004311
21    0.000807
22         NaN
dtype: float64

In [9]:
##### Change in Net Financial Assets (ChNFA)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at','ivst','ivao','dltt','dlc','pstk']]

df2.loc[max(df2.index)+1, :] = None

chnfa = ((df2['ivst'].fillna(0) + df2['ivao'].fillna(0) - df2['dltt'] - df2['dlc'].fillna(0) - df2['pstk'].fillna(0)) - (df2['ivst'].fillna(0) + df2['ivao'].fillna(0) - df2['dltt'] - df2['dlc'].fillna(0) - df2['pstk'].fillna(0)).shift(1))/ df2['at'].shift(1)

chnfa


0          NaN
1     0.089190
2    -0.108720
3     0.045023
4    -0.087122
5    -0.168847
6     0.048374
7     0.035573
8     0.049939
9     0.039311
10   -0.116368
11    0.060400
12    0.013505
13    0.072547
14    0.040429
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.000000
20   -0.012325
21    0.004474
22         NaN
dtype: float64

In [16]:
##### Change in Long-Term Investments (ChLTI)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at','ivao']]

df2.loc[max(df2.index)+1, :] = None

chlti = (df2['ivao'].fillna(0) - df2['ivao'].shift(1))/ df2['at'].shift(1)

chlti


0          NaN
1     0.000000
2     0.000000
3     0.026785
4     0.005512
5    -0.035523
6     0.000000
7     0.000000
8     0.000000
9     0.000000
10    0.000000
11    0.000000
12    0.016473
13    0.000000
14   -0.013338
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.000000
20    0.000000
21    0.000000
22         NaN
dtype: float64

In [19]:
##### Change in Common Equity (ChCE)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at','ceq']]

df2.loc[max(df2.index)+1, :] = None

chce = (df2['ceq'] - df2['ceq'].shift(1))/ df2['at'].shift(1)

chce


0          NaN
1     0.571176
2     0.016181
3    -0.073135
4    -0.077404
5     0.062151
6    -0.021997
7     0.046613
8     0.050967
9    -0.001423
10   -0.099574
11    0.199872
12    0.204540
13    0.051876
14    0.470404
15    0.156616
16    0.141318
17    0.032504
18   -0.002022
19   -0.169925
20    0.095835
21    0.016584
22         NaN
dtype: float64

In [21]:
##### Change in Financial Liabilities (ChFL)
df2 = df.loc[df['gvkey'] == '001166']
df2 = df2[['date','at','dltt','dlc','pstk']]

df2.loc[max(df2.index)+1, :] = None

chfl = (df2['dltt']+df2['dlc'].fillna(0)+df2['pstk'].fillna(0) - (df2['dltt']+df2['dlc'].fillna(0)+df2['pstk'].fillna(0)).shift(1)) / df2['at'].shift(1)

chfl


0          NaN
1    -0.102610
2     0.108720
3    -0.018230
4     0.092631
5     0.133318
6    -0.048375
7    -0.035578
8    -0.049939
9    -0.039311
10    0.116368
11   -0.060400
12    0.002968
13   -0.072547
14   -0.053766
15    0.000000
16    0.000000
17    0.000000
18    0.000000
19    0.000000
20    0.012325
21   -0.004474
22         NaN
dtype: float64